<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Imports" data-toc-modified-id="Imports-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Imports</a></span></li><li><span><a href="#Grid-information" data-toc-modified-id="Grid-information-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Grid information<a class="anchor" id="0_Grid_information" rel="nofollow"></a></a></span></li><li><span><a href="#Definitions" data-toc-modified-id="Definitions-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Definitions</a></span></li><li><span><a href="#Import-grid" data-toc-modified-id="Import-grid-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Import grid</a></span></li><li><span><a href="#Plots" data-toc-modified-id="Plots-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Plots</a></span></li><li><span><a href="#Capacities" data-toc-modified-id="Capacities-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Capacities</a></span><ul class="toc-item"><li><span><a href="#Charging-+-feedin" data-toc-modified-id="Charging-+-feedin-6.1"><span class="toc-item-num">6.1&nbsp;&nbsp;</span>Charging + feedin</a></span></li><li><span><a href="#curtailment" data-toc-modified-id="curtailment-6.2"><span class="toc-item-num">6.2&nbsp;&nbsp;</span>curtailment</a></span></li><li><span><a href="#Slacks" data-toc-modified-id="Slacks-6.3"><span class="toc-item-num">6.3&nbsp;&nbsp;</span>Slacks</a></span></li><li><span><a href="#Energy" data-toc-modified-id="Energy-6.4"><span class="toc-item-num">6.4&nbsp;&nbsp;</span>Energy</a></span></li><li><span><a href="#Charging" data-toc-modified-id="Charging-6.5"><span class="toc-item-num">6.5&nbsp;&nbsp;</span>Charging</a></span></li></ul></li><li><span><a href="#scenario-comparison" data-toc-modified-id="scenario-comparison-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>scenario comparison</a></span><ul class="toc-item"><li><span><a href="#Reinforcement-costs" data-toc-modified-id="Reinforcement-costs-7.1"><span class="toc-item-num">7.1&nbsp;&nbsp;</span>Reinforcement costs</a></span></li><li><span><a href="#Flexbands" data-toc-modified-id="Flexbands-7.2"><span class="toc-item-num">7.2&nbsp;&nbsp;</span>Flexbands</a></span><ul class="toc-item"><li><span><a href="#Heat-pump" data-toc-modified-id="Heat-pump-7.2.1"><span class="toc-item-num">7.2.1&nbsp;&nbsp;</span>Heat pump</a></span></li></ul></li><li><span><a href="#EV" data-toc-modified-id="EV-7.3"><span class="toc-item-num">7.3&nbsp;&nbsp;</span>EV</a></span></li><li><span><a href="#Plot-reference-+-opt-charging" data-toc-modified-id="Plot-reference-+-opt-charging-7.4"><span class="toc-item-num">7.4&nbsp;&nbsp;</span>Plot reference + opt charging</a></span></li><li><span><a href="#Plot-energy-level-+-charging" data-toc-modified-id="Plot-energy-level-+-charging-7.5"><span class="toc-item-num">7.5&nbsp;&nbsp;</span>Plot energy level + charging</a></span></li></ul></li></ul></div>

# Imports

In [ ]:
######## LOBAFLEX ###########
# from lobaflex.analysis import analysis
# from lobaflex.tools.draw import *
from lobaflex.opt.feeder_extraction import get_flexible_loads
from lobaflex.tools.tools import get_config, get_files_in_subdirs
from lobaflex.opt.dispatch_integration import integrate_dispatch
from edisgo.edisgo import import_edisgo_from_files

######## DIRS ##########
from lobaflex import config_dir, data_dir, results_dir
######## DATA ###########
import re
import numpy as np
import pandas as pd
from pathlib import Path
######## PLOTTING ###########
import matplotlib
# matplotlib.use('TkAgg')
# %matplotlib inline
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib.ticker import ScalarFormatter
import seaborn as sns

# # set seaborn style
# sns.set()
# plotly
import plotly.express as px
import plotly.graph_objs as go
from plotly.subplots import make_subplots
import plotly.io as pio


from lobaflex.tools.plots import (get_all_attribute_values_for_keyword,
                                  plot_all_attributes_for_keyword,
                                  plot_optimized_dispatch,
                                  heatmap_energy_level,
                                  plot_scenario_potential,
                                  plot_compare_optimization_to_reference,
                                  plot_flex_capacities
                                 )
# comment to make plots interactive
# pio.renderers.default = "svg"

# Deactivate warning
import logging
logger = logging.getLogger("edisgo.network.topology")
logger.setLevel(logging.ERROR)


In [ ]:
pio.renderers.default = "svg"

In [ ]:
cfg_o = get_config(path=config_dir / ".opt.yaml")
# cfg_g = get_config(path=config_dir / ".grids.yaml")

# Grid information<a class="anchor" id="0_Grid_information"></a>

# Definitions

In [ ]:
grid_path = Path(grid_path) 
# base_objective =  grid_path.parent.name
# results_path = results_dir / run_id / str(grid_id) / "potential" / base_objective
# list_of_results = get_files_in_subdirs(path=results_path, pattern="*.csv")

df_observation_periods = pd.read_csv(results_dir / "observation_periods.csv",
                                     header=[0,1], index_col=0).loc[:,str(grid_id)]

# Import grid

In [ ]:
edisgo_obj = import_edisgo_from_files(
    grid_path / "minimize_loading" / "mvgd",
    import_topology=True,
    import_timeseries=True,
    import_heat_pump=True,
    import_electromobility=True,
)

# Plots

# Capacities

In [ ]:
plot_flex_capacities(edisgo_obj)

## Charging + feedin

In [ ]:
plot_optimized_dispatch(edisgo_obj, timeframe=df_observation_periods[period])

## curtailment

In [ ]:
plot_all_attributes_for_keyword(results_path=grid_path / "potential" / "minimize_loading",
                                keyword="curtailment",
                                timeframe=df_observation_periods[period])

## Slacks

In [ ]:
plot_all_attributes_for_keyword(results_path=grid_path / "potential" / "minimize_loading",
                                keyword="slack",
#                                timeframe=df_observation_periods[period]
                               )

## Energy

In [ ]:
plot_all_attributes_for_keyword(results_path=grid_path / "potential" / "minimize_loading",
                                keyword=f"{grid_id}_energy_level",
                                edisgo_obj=edisgo_obj,
                                objectives=["minimize_energy_level"],
                               timeframe=df_observation_periods[period])

In [ ]:
plot_all_attributes_for_keyword(results_path=grid_path / "potential" / "minimize_loading",
                                keyword=f"{grid_id}_energy_level",
                                edisgo_obj=edisgo_obj,
                                objectives=["maximize_energy_level"],
                               timeframe=df_observation_periods[period])

min max energy level - what happens at peaks? feedin und demand plots?

## Charging

charging ev: hüllkurve upper lower opacitiy und dann opt ergebnis dazwischen
hp: same

annotation for max min grid power: no tes energy constraints

In [ ]:
plot_all_attributes_for_keyword(results_path=grid_path / "potential" / "minimize_loading",
                                keyword="charging",
                                edisgo_obj=edisgo_obj,
                                objectives=["minimize_grid_power"],
                                timeframe=df_observation_periods["potential"])

In [ ]:
plot_all_attributes_for_keyword(results_path=grid_path / "potential" / "minimize_loading",
                                keyword="charging",
                                edisgo_obj=edisgo_obj,
                                objectives=["maximize_grid_power"],
                                timeframe=df_observation_periods["potential"])

- high initial charging bei step 1
    - hp
    - tes
- Speicher werden direkt voll gemacht

- konstantes laden der TES wegen verlusten
- warum Einbruch am Ende?

# scenario comparison

## Reinforcement costs

In [ ]:
scenarios = [f"{i}_pct_reinforced" for i in [20, 40, 60]]

edisgo_paths = [grid_path / "scenarios" / scenario / "mvgd" for scenario in scenarios]
edisgo_paths += [grid_path / "reference" / "reinforced"]
edisgo_paths += [grid_path / "minimize_loading" / "reinforced"]

for edisgo_path in edisgo_paths:
    edisgo_obj = import_edisgo_from_files(
        edisgo_path,
        import_topology=False,
        import_timeseries=False,
        import_heat_pump=False,
        import_electromobility=False,
        import_results=True
    )

    costs = edisgo_obj.results.grid_expansion_costs['total_costs'].sum()
    print(f"{edisgo_path.parent.name}: {int(costs):,} k €")

## Flexbands

### Heat pump

In [ ]:
path = results_dir / run_id / str(grid_id) / "potential"
objectives = ["maximize_grid_power", "minimize_grid_power"]
keyword = "charging"
technology = "hp_el"
timeframe = df_observation_periods["potential"]

plot_scenario_potential(path, objectives, keyword, technology, timeframe, title=None)

In [ ]:
path = results_dir / run_id / str(grid_id) / "potential"
objectives = ["maximize_energy_level", "minimize_energy_level"]
keyword=f"{grid_id}_energy_level"
technology = "tes"
timeframe = df_observation_periods["potential"]

plot_scenario_potential(path, objectives, keyword, technology, timeframe, title=None)

## EV

In [ ]:
path = results_dir / run_id / str(grid_id) / "potential"
objectives = ["maximize_grid_power", "minimize_grid_power"]
keyword = "charging"
technology = "ev"
timeframe = df_observation_periods["potential"]

plot_scenario_potential(path, objectives, keyword, technology, timeframe, title=None)

In [ ]:
path = results_dir / run_id / str(grid_id) / "potential"
objectives = ["maximize_energy_level", "minimize_energy_level"]
keyword = f"{grid_id}_energy_level"
technology = "ev"
timeframe = df_observation_periods["potential"]

plot_scenario_potential(path, objectives, keyword, technology, timeframe, title=None)

## Plot reference + opt charging

In [ ]:
plot_compare_optimization_to_reference(grid_path, timeframe=df_observation_periods['min_load'])

## Plot energy level + charging